# Modélisation

In [1]:
from google.colab import drive
drive.mount('/content/drive') 
import os
os.chdir("/content/drive/MyDrive/Compet_Kaggle/Space_titanic")

Mounted at /content/drive


In [81]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from sklearn.model_selection import cross_val_score
import optuna
from time import time
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline


#### Installation LGBM et xgboost

In [41]:
!pip install lightgbm 
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 8.9 MB/s 
     |████████████████████████████████| 81 kB 11.5 MB/s 
     |████████████████████████████████| 209 kB 75.2 MB/s 
     |████████████████████████████████| 78 kB 9.6 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 147 kB 59.5 MB/s 
     |████████████████████████████████| 112 kB 59.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=3d1537310833c781985d1cfbc656e5279344aec4a5c3f8054b9017f25f64295a
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


## All feature without imputation resaerch like classic imputation of missing values (Mean, median, 0 ... ) 
Avec passage au logarithme de d_[['RoomService','FoodCourt','ShoppingMall','Spa']]
### Sans se soucier des valeurs aberrantes

In [61]:
def prepro_1(d_,sub=False):

  """ 
  Preprocessing avec sub True ou pas selon si cest pour la soumission donc les données test ou non 

  """

  inf_cabin=list(d_.Cabin.dropna().apply(lambda x:x.split('/')).values)
  d_=d_.join(pd.DataFrame(inf_cabin,columns=['cab1','cab2','cab3'],index=d_.Cabin.dropna().index))
  d_.drop('Cabin',axis=1,inplace=True)
  d_[['RoomService','FoodCourt','ShoppingMall','Spa']]=np.log(d_[['RoomService','FoodCourt','ShoppingMall','Spa']]+1)
  d_.cab2=d_.cab2.dropna().astype('int')
  for col in ['VIP','CryoSleep']:
    d_[col] = d_[col].apply(lambda x: 1 if x else 0)
  if 'Transported' in d_.columns:
    d_.drop('Transported',axis=1,inplace=True)
  if sub==True:
    d_.drop(['Name'],axis=1,inplace=True)
  else: 
    d_.drop(['PassengerId','Name'],axis=1,inplace=True)
  return d_

In [64]:
train = pd.read_csv('train.csv')
X_train,y_train = prepro_1(train), train.Transported.apply(lambda x: 1 if x else 0)

In [65]:
encod=Pipeline(steps=[('imputer',SimpleImputer(strategy="constant",fill_value='Not Known')),
                       ('encod',OrdinalEncoder())])
imputer=SimpleImputer(strategy="constant",)
pip_cat=Pipeline(steps=[('imput_cat',SimpleImputer(strategy="constant",fill_value='other')),
                        ('encod',encod)])


preprocessor = ColumnTransformer(
        transformers=[
            ("cat", pip_cat, X_train.select_dtypes('object').columns),
            ("num",imputer,X_train.select_dtypes(exclude='object').columns)
        
        ]
    )

In [54]:
X_train.select_dtypes('object').VIP

,HomePlanet,CryoSleep,Destination,VIP,cab1,cab3
0,Europa,False,TRAPPIST-1e,False,B,P
1,Earth,False,TRAPPIST-1e,False,F,S
2,Europa,False,TRAPPIST-1e,True,A,S
3,Europa,False,TRAPPIST-1e,False,A,S
4,Earth,False,TRAPPIST-1e,False,F,S
...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,True,A,P
8689,Earth,True,PSO J318.5-22,False,G,S
8690,Earth,False,TRAPPIST-1e,False,G,S
8691,Europa,False,55 Cancri e,False,E,S


In [51]:
X_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,cab1,cab2,cab3
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0.0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0.0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0.0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0.0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1.0,S


In [67]:

models = [LGBMClassifier(),XGBClassifier(),RandomForestClassifier()]
res_cv = {}

for mod in models:
    t0=time()
    pipeline = Pipeline(steps=[('prepro',preprocessor),('clf',mod)])
    cv = cross_validate(pipeline, X_train, y_train, cv=8,scoring=('accuracy', 'f1'))
    res_cv[str(mod)[:-2]]=cv
    print(f'Temps cv pour {str(mod)[:-2] :-<70} {time()-t0} secondes')




Temps cv pour LGBMClassifier-------------------------------------------------------- 2.135944128036499 secondes
Temps cv pour XGBClassifier--------------------------------------------------------- 4.056429386138916 secondes
Temps cv pour RandomForestClassifier------------------------------------------------ 8.465834140777588 secondes


In [71]:
res_cv[list(res_cv.keys())[0]]

{'fit_time': array([0.2745595 , 0.25883627, 0.24419117, 0.24248815, 0.2442863 ,
        0.25801134, 0.23125768, 0.22903681]),
 'score_time': array([0.02060366, 0.01513577, 0.01466584, 0.01507401, 0.01608801,
        0.01506591, 0.01513267, 0.01506376]),
 'test_accuracy': array([0.7976081 , 0.7424103 , 0.74609016, 0.76632935, 0.8150874 ,
        0.82780847, 0.80018416, 0.74953959]),
 'test_f1': array([0.79742173, 0.70588235, 0.73962264, 0.7806563 , 0.82746781,
        0.8343667 , 0.80109991, 0.68372093])}

In [79]:
for mod in res_cv.keys(): 
  print(f'+-+-Affichage pour {mod} : ')
  for re in res_cv[mod].keys():
    print(f'    - {re} pour {str(mod) :-<70} {res_cv[mod][re].mean()} ')

+-+-Affichage pour LGBMClassifier : 
    - fit_time pour LGBMClassifier-------------------------------------------------------- 0.24783340096473694 
    - score_time pour LGBMClassifier-------------------------------------------------------- 0.015853703022003174 
    - test_accuracy pour LGBMClassifier-------------------------------------------------------- 0.7806321909186247 
    - test_f1 pour LGBMClassifier-------------------------------------------------------- 0.7712797969337517 
+-+-Affichage pour XGBClassifier : 
    - fit_time pour XGBClassifier--------------------------------------------------------- 0.4919322431087494 
    - score_time pour XGBClassifier--------------------------------------------------------- 0.012831509113311768 
    - test_accuracy pour XGBClassifier--------------------------------------------------------- 0.7940919471876742 
    - test_f1 pour XGBClassifier--------------------------------------------------------- 0.7976039895488443 
+-+-Affichage pour Ran

In [82]:
def objective(trial):
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 70, 300)
    rf_max_depth = trial.suggest_int("rf_max_depth", 1, 6)
    
    encod=Pipeline(steps=[('imputer',SimpleImputer(strategy="constant",fill_value='Not Known')),
                          ('encod',OrdinalEncoder())])
    imputer=SimpleImputer(strategy="constant",)
    pip_cat=Pipeline(steps=[('imput_cat',SimpleImputer(strategy="constant",fill_value='other')),
                            ('encod',encod)])


    preprocessor = ColumnTransformer(
            transformers=[
                ("cat", pip_cat, X_train.select_dtypes('object').columns),
                ("num",imputer,X_train.select_dtypes(exclude='object').columns)
            
            ]
        )
    
    mod = XGBClassifier(n_estimators=rf_n_estimators,max_depth=rf_max_depth)
    pipeline = Pipeline(steps=[('prepro',preprocessor),('clf',mod)])
   

    model = XGBClassifier(
        n_estimators=rf_n_estimators,
        max_depth=rf_max_depth
        
    )

    score = cross_val_score(pipeline, X_train , y_train, cv=8,scoring='accuracy')
    accuracy = score.mean()
    return accuracy
study = optuna.create_study(direction="maximize")
study.optimize(objective,timeout=600)
print(study.best_trial)

[I 2022-10-03 23:35:10,989] A new study created in memory with name: no-name-eb2abdc1-d4d7-462e-adb2-f1d196605128
[I 2022-10-03 23:35:21,003] Trial 0 finished with value: 0.7813216338749764 and parameters: {'rf_n_estimators': 196, 'rf_max_depth': 4}. Best is trial 0 with value: 0.7813216338749764.
[I 2022-10-03 23:35:26,469] Trial 1 finished with value: 0.7789084246943198 and parameters: {'rf_n_estimators': 86, 'rf_max_depth': 5}. Best is trial 0 with value: 0.7813216338749764.
[I 2022-10-03 23:35:32,963] Trial 2 finished with value: 0.7884535723543434 and parameters: {'rf_n_estimators': 236, 'rf_max_depth': 2}. Best is trial 2 with value: 0.7884535723543434.
[I 2022-10-03 23:35:49,547] Trial 3 finished with value: 0.7754560213539894 and parameters: {'rf_n_estimators': 294, 'rf_max_depth': 5}. Best is trial 2 with value: 0.7884535723543434.
[I 2022-10-03 23:35:55,175] Trial 4 finished with value: 0.7889136598440298 and parameters: {'rf_n_estimators': 287, 'rf_max_depth': 1}. Best is tr

FrozenTrial(number=64, values=[0.7947813901440259], datetime_start=datetime.datetime(2022, 10, 3, 23, 41, 17, 816040), datetime_complete=datetime.datetime(2022, 10, 3, 23, 41, 21, 540226), params={'rf_n_estimators': 94, 'rf_max_depth': 3}, distributions={'rf_n_estimators': IntDistribution(high=300, log=False, low=70, step=1), 'rf_max_depth': IntDistribution(high=6, log=False, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=64, state=TrialState.COMPLETE, value=None)


In [85]:
study.best_params

{'rf_n_estimators': 94, 'rf_max_depth': 3}

In [87]:
mod = XGBClassifier(n_estimators=94,max_depth=3)
pipeline = Pipeline(steps=[('prepro',preprocessor),('clf',mod)])

In [88]:
pipeline

Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imput_cat',
                                                                   SimpleImputer(fill_value='other',
                                                                                 strategy='constant')),
                                                                  ('encod',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(fill_value='Not '
                                                                                                             'Known',
                                                                                                  strategy='constant')),
                                                                                   ('encod',
            

In [89]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imput_cat',
                                                                   SimpleImputer(fill_value='other',
                                                                                 strategy='constant')),
                                                                  ('encod',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(fill_value='Not '
                                                                                                             'Known',
                                                                                                  strategy='constant')),
                                                                                   ('encod',
            

In [90]:
pipeline.predict(X_train)

array([1, 0, 0, ..., 1, 0, 1])

In [92]:
test = pd.read_csv('test.csv')

In [96]:
test['Transported']=pipeline.predict(prepro_1(test,sub=True))
test.Transported = test.Transported.apply(lambda x:True if x==1 else False)

In [98]:
test[['PassengerId','Transported']].to_csv('sub_2.csv',index=False)

### Avec gestion des valeurs aberrantes

### Optuna pour optimisation hyperparamètres

In [ ]:
"""
Inspiration afin de maximiser notre fonction coût

"""

def objective(trial):

    rf_n_estimators = trial.suggest_int("rf_n_estimators", 80, 200)
    rf_max_depth = trial.suggest_int("rf_max_depth", 1, 6)
    
    model = GradientBoostingRegressor(
        n_estimators=rf_n_estimators,
        max_depth=rf_max_depth
        
    )

    score = cross_val_score(model, x_quad , ytrain, cv=8,scoring='r2')
    accuracy = score.mean()
    return accuracy
study = optuna.create_study(direction="maximize")
study.optimize(objective,timeout=600)
print(study.best_trial)

## With research of imputation 

##Feature engineering (polynomial of floatting features) 

## Other approach